In [ ]:
import os
import glob
import shutil
from pathlib import Path
import csv
import pandas as pd
Data_dir = 'Data'+os.sep+"MIX"+os.sep

In [ ]:
def get_numerical_value(my_text):
    """Get numerical value from text string"""
    import re
    feet_units=['F','FT','FEET']
    meter_units=['M','METER','METERS']
    Km_units=['KM','KILOMETER','KILOMETERS']
    unit_used = re.findall('[a-zA-Z]+', my_text)[0]
    numeric_value = float(my_text.split(unit_used)[0])
    
    if unit_used.upper() in feet_units:
        return (int(numeric_value))
    
    elif unit_used.upper() in meter_units:
        return(int(numeric_value*3.28084))
    
    elif unit_used.upper() in Km_units:
        return(int((numeric_value*1000)*3.28084))
    else:
        print("Error: in function: (get_numerical_value) as the unit used is invalid")
    

In [ ]:
def get_numerical_value_in_m(my_text):
    """Get numerical value from text string"""
    import re
    feet_units=['F','FT','FEET']
    meter_units=['M','METER','METERS']
    Km_units=['KM','KILOMETER','KILOMETERS']
    unit_used = re.findall('[a-zA-Z]+', my_text)[0]
    numeric_value = float(my_text.split(unit_used)[0])

    if unit_used.upper() in feet_units:
        return (int(numeric_value * 0.3048))   #length of road in a frame in meters
    elif unit_used.upper() in meter_units:
        return(int(numeric_value))
    elif unit_used.upper() in Km_units:
        return(int(numeric_value/1000))
    else:
        print("Error: in function: (get_numerical_value) as the unit used is invalid")

In [ ]:
roads_list=glob.glob(Data_dir+'DI Khan'+os.sep+'roads'+os.sep+'*')

In [ ]:
roads_details = []
for road in roads_list:
    road_name = road.split(os.sep)[-1]
    print(road_name)
    segments_srvyList=glob.glob(os.path.join(road, '**', '*.xlsx'), recursive=True)
    segments_crackList=glob.glob(os.path.join(road, '**', 'Area_Count*'), recursive=True)


    for seg_srvy in segments_srvyList:
        seg_name = seg_srvy.split(os.sep)[6]
        seg_dir  = os.path.dirname(seg_srvy)
        areacount = glob.glob(seg_dir+os.sep+'Area_Count*.csv')[0]
        
        rd_areacount =pd.read_csv(areacount)
        rd_survey =pd.read_excel(seg_srvy)
        start_pt = rd_survey.iloc[11,1]

        seg_len = get_numerical_value_in_m(rd_survey.iloc[7,1])
        seg_PCI = rd_areacount["PCI"][0]
 
        seg_width = get_numerical_value(rd_survey.iloc[8,1])
        seg_start = rd_survey.iloc[11,1]
        seg_end = rd_survey.iloc[12,1]
        
        if seg_PCI > 70:
            roads_details.append([road_name,seg_name,seg_len,seg_width,seg_start,seg_end,seg_PCI])


    
df_roads=pd.DataFrame(roads_details,columns=["Road","Segments","Length(m)","Width","Start Point","End Point","PCI"])

# Save to CSV file
df_roads.to_csv(Data_dir+'Filtered Segments.csv', index=False)       